# 1 - Sequence to Sequence Learning with Neural Networks

Seq2Seq 는 기본적으로 어떠한 sequence 를 다른 형태로 변형하는 머신러닝 모델을 의미한다. 이는 한국어를 영어로 바꾸는 translation 이 될 수도 있고, 어떤 sequence 를 받아서 그것에 대한 요약(summarization) 으로 바꿔주는 것도 포함한다.

해당 내용은 2014년도 [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) 논문의 내용의 컨셉을 구현한 것임.

## Introduction

일반적으로 seq2seq 모델은 *encoder-decoder* 의 형태를 띄고 있는데, 가장 기본적인 형태로는 RNN(*Recurrent Neural Network*)이 있다. RNN 은 입력과 출력의 길이를 서로 다르게 설계할 수 있어서 다양한 용도로 사용할 수 있다.

<p align="center"><img src="../asset/1(1).png"></p>

- 일 대 다 구조는 하나의 이미지 입력에 대해 사진의 제목 또는 사진에 대한 설명을 출력하는 image captioning 태스크에 이용될 수 있다. 
- 다 대 일 구조는 입력으로 주어진 문서에 대한 분류를 수행하는 sentiment classification 과 같은 데 사용될 수 있다.
- 다 대 다 구조는 주어진 문장에 대해 대답하거나, 번역을 수행하거나, 요약하는 문장을 만드는 등의 태스크에 사용될 수 있다.

</br>
<p align="center"><img src="../asset/1(2).png"></p>

위의 그림은 translation 을 수행하는 RNN 구조의 예시인데, input 문장이 <span style='color:#ffd33d'>embedding layer(노란색)</span>를 거친 후에 <span style='color:#008000'>encoder</span>의 입력으로 들어가게 된다. 이때 입력 문장의 앞 뒤에는 *start of sequence* (`SOS`)와 *end of sequence* (`EOS`)가 추가된다. <span style='color:#008000'>Encoder</span> 에서는 각 time-step 마다 input 으로 현재 time-step 의 embedding $e(x_t)$와 previous time step 의 hidden state $h_{t-1}$ 이 들어오고, output 으로는 새로운 현 시점의 hidden state $h_t$ 를 내뱉는다.

보통 initial hidden state $h_0$ 은 zero-initial 또는 학습된 parameter로 구성한다. input 문장의 마지막 $x_T$ 가 RNN 을 거치게 되면 최종 hidden state 인 $h_T$ 가 나오게 되고, 이를 input 문장의 최종 context vector ($z$) 로 사용하게 된다.

다음으로 얻어진 context vector $z$ 를 통해 target 문장을 <span style='color:#0000FF'>decode</span> 하는데, 각 time-step 마다 <span style='color:#0000FF'>decoder</span>의 input 으로 현재 단어에 대한 embedding $d(y_t)$과 전 단계의 hidden state인 $s_{t-1}$이 들어간다. 이때 decoder의 initial hidden state 는 앞에서 얻어진 context vecotr $z = h_T$ 이다. 

얻어진 hidden state $s_t$ 를 통해 <span style='color:#FF00FF'>Linear</span> layer 에 넣어서, 예측한 다음 단어 $\hat{y} = f(s_t)$ 를 얻는다

이런 과정을 반복하면서, 학습 때에는 target 문장의 길이를 알고 있으므로, 해당 문장의 길이가 채워질 때까지, inference 때에는 `EOS` 토큰이 생성될 때까지, 또는 특정 개수만큼 단어가 생성될 때까지 수행하게 된다.

## Data Preparation

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

Reproducible 을 위해 random seed 고정

In [2]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Tokenizer 생성

Tokenizer 는 NLP 의 핵심 구성 요소 중 하나로, 오직 입력된 텍스트를 신경망에서 처리할 수 있는 형태의 데이터로 변환하는 하나의 목적을 가지고 있다. 즉, string 형태의 데이터를 모델이 처리할 수 있게 숫자로 변환해주는 역할을 수행한다.

workspace 경로에 가서 아래를 통해 각각 독일어, 영어를 위한 tokenizer 를 다운받는다.
```
python -m spacy download en_core_web_sm

python -m spacy download de_core_news_sm
```

In [5]:
import en_core_web_sm, de_core_news_sm

spacy_de = de_core_news_sm.load()
spacy_en = en_core_web_sm.load()

선언된 tokenizer 함수는 torchtext 를 통해 input 문장을 `string` 형태로 받아서, token 들로 이루어진 문장을 반환하게 된다.

아래 코드에서 독일어에 대해서 token 들을 역정렬해주는데, 이는 해당 논문에서 순서를 reverse 하게 해주는 것이 data 내부의 short term dependency 를 더해줄 수 있다고 언급했기 때문이다.

In [7]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

torchtext의 `Field` 는 데이터의 처리방법을 설정해줄 수 있다.

- `tokenize` : tokenize를 수행할 함수를 지정
- `init_token, eos_token` : `Field`에서 데이터 처리 시 SOS, EOS를 추가할 수 있도록
- `lower` : 모든 단어를 소문자로 바꿔줌

이밖에도 
`pad_token`, `unk_token` 등 다양한 토큰을 지정해줄 수도 있고,

`pad_first`, `batch_first`, `truncate_first` 등의 다양한 옵션이 있음.

자세한 사항은 해당 [링크](https://torchtext.readthedocs.io/en/latest/data.html#fields)에서 확인할 수 있다.

In [8]:
SRC = Field(
    tokenize=tokenize_de,
    init_token='<sos>',
    eos_token='<eos>',
    lower=True
)

TRG = Field(
    tokenize=tokenize_en,
    init_token='<sos>',
    eos_token='<eos>',
    lower=True
)

사용할 데이터 셋은 `Multi30k dataset` 으로 영어, 독일어, 프랑스어로 이루어진 문장들의 쌍으로 구성되어 있는 데이터 셋이다. 각 문장은 약 12개 정도의 단어로 구성되어 있다.

In [9]:
train_data, valid_data, test_data = Multi30k.splits(
    exts=('.de', '.en'),
    fields=(SRC, TRG)
)

print(f'Number of training data : {len(train_data.examples)}')
print(f'Number of valid data : {len(valid_data.examples)}')
print(f'Number of test data : {len(test_data.examples)}')

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 495kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 86.7kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 123kB/s] 


Number of training data : 29000
Number of valid data : 1014
Number of test data : 1000


파이썬 내장 함수 `vars()` 를 통해 데이터가 어떻게 생겼는지 확인

- `vars()` : 모듈, 클래스, 인스턴스 객체 (__dict__ 속성을 가지는 개체)에 대해 dictionary 형태로 반환해주는 함수

In [10]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


다음으로는 `vocabulary` 를 사용하여 각 token과 결부되는 index 를 만들어 준다. 

`min_freq` 를 통해 해당 단어가 몇 번 이상 등장해야 vocabulary 로 볼 것인지 정해줄 수 있고, 이보다 적게 등장하면 `<unk>` 토큰으로 변환된다.

In [11]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

print(f'Unique tokens in source vocab : {len(SRC.vocab)}')
print(f'Unique tokens in target vocab : {len(TRG.vocab)}')

Unique tokens in source vocab : 7854
Unique tokens in target vocab : 5893


마지막으로는 batch 학습을 수행할 수 있게 *iterator* 를 만들어 준다. 

batch를 구성할 때는, source 문장과 target 문장의 길이가 같아야하고, 이를 위해서는 padding을 수행해야한다.


이 때, `BucketIterator`를 사용해주면, 데이터들을 통해 batch 를 형성할 때, 가장 padding 이 적게 될 수 있도록 자동으로 batch 를 구성해준다.

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    datasets=(train_data, valid_data, test_data),
    batch_size=batch_size,
    device=device
)

## Building the Seq2Seq Model

이제 Encoder, Decoder, 그리고 그 둘을 통합한 seq2seq 모델을 캡슐화(encapsulate)한다.

### Encoder

인코더는 2-layer LSTM으로 구성함. 

우선 input 문장 $X$는 embedding layer 를 거쳐 hidden state $H= \{ h_1, h_2, h_3, ..., h_T \}$ 가 된다.

Multi-layer RNN 은 각 layer 마다 initial hidden layer $h^l_0$ 이 존재하고, 따라서 context vector 도 각각 $z_l$ 로 표현된다.

LSTM은 RNN과 비슷하게, time-step 마다 hidden state를 받아서 새로운 hidden state를 내뱉는데, 추가적으로 time-step 마다 *cell state* 라는 것도 받고 내뱉게 된다.

그림은 아래와 같다.

<p align="center"><img src="../asset/1(3).png"></p>

</br></br>

`Encoder` 모듈에는 다음과 같은 인자들이 들어가게 된다.

`super().__init__()` : boilerplate code 라고도 하는데, 변화 없이 여러 군데에서 반복되는 코드를 말한다. 아래에서는 `nn.Module` (parent class) 의 `__init__` 함수를 불러 와서 `Encoder` 에서 쓰일 수 있도록 하는 것임.

`input_dim` : Encoder 의 input으로 들어오는 one-hot vector 의 차원을 의미한다. 앞에서 봤던 vocabulary 의 크기와 동일하다

`emb_dim` : Embedding layer 의 차원을 의미한다. 즉, input 인 one-hot encoding을 받아서 몇차원의 dense vector로 만들어 줄 것인가에 대한 것이다.

`hid_dim` : hidden state 와 cell state 의 차원

`n_layers` : RNN 구조의 몇 개의 층이 있는지

`dropout` : dropout, 즉 regularization 의 정도. 흔히 쓰는 dropout layer와는 다르게, hideen state 들이 layer를 거치면서 올라올 때 특정 확률로 dropout 되는 것이다. 즉 위의 그림에서 가로 방향은 유지가 되지만 세로 방향의 forward 는 dropout 되는 것이다

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        
        '''
        src     - [src_len, batch_size] 
        '''

        embedded = self.dropout(self.embedding(src)) # embedded = [src_len, batch_size, emb_dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        # outputs = [src_len, batch_size, hid_dim * n_directions]
        # hidden, cell = [n_layers * n_directions, batch_size, hid_dim]
        
        return hidden, cell

### Decoder

<p align="center"><img src="../asset/1(4).png"></p>

`Decoder`에서는 전 단계의 hidden_state와 cell_state를 받아서 현 단계의 embedded token과 함께 새로운 현 단계의 hidden, cell state를 생성하게 된다. 

`Decoder`의 initial hidden, cell state 는 앞의 `Encoder`의 final hidden, cell state 이다.

`Decoder`는 얻어진 vector로 단어를 예측해야하므로, `Encoder`와 달리 output의 차원은 target의 vocabulary 크기와 같다.

In [26]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden ,cell):

        '''
        input       - [batch_size] 
        hidden      - [n_layers * n_directions, batch_size, hid_dim]
        cell        - [n_layers * n_directions, batch_size, hid_dim]

        여기서 Decoder 에서 n_direction 은 항상 1 이다. 
        '''
        input = input.unsqueeze(0) # [batch_size] -> [1, batch_size]

        embedded = self.dropout(self.embedding(input)) # [1, batch_size, emb_dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        # output = [seq_len, batch_size, hid_dim * n_directions]
        # hidden = [n_layers * n_directions, batch_size, hid_dim]
        # cell = [n_layers * n_directions, batch_size, hid_dim]
        # 이 때, Decoder에서 seq_len 과 n_directions 는 항상 1 이다.

        prediction = self.fc_out(output.squeeze(0)) # [batch_size, output_dim]

        return prediction, hidden, cell

### Seq2Seq

최종적으로 구현할 Seq2seq 모델은 다음과 같은 역할을 수행해야 한다.

- input(source) 문장을 받는다
- Encoder 를 통해 context vector 를 생성한다
- Decoder 를 통해 output(target) 문장을 예측한다.

<p align="center"><img src="../asset/1(5).png"></p>

**Teacher Forcing**

구현한 모델의 `forward` 부분에서 `teacher_forcing_ratio`를 사용하는데 이는, target word(ground truth)를 디코더의 다음 입력으로 넣어주는 기법을 의미한다.



디코더는 한 단어, 한 단어를 순차적으로 생성하기 때문에 어느 부분에서 잘못된 예측이 발생하면, 그 뒤부터는 모두 잘못된 예측일 확률이 높아진다. 

이로 인해 초기 학습 속도가 굉장히 낮아지기 때문에 이를 개선하고자 수행하는 기법이다.

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # 사실 같을 필요는 없지만 본 구현에서는 같다고 가정했기 때문에 이를 assert 해야함.

        assert encoder.hid_dim == decoder.hid_dim, 'Hidden dimensions of encoder and decoder should be euqal'
        assert encoder.n_layers == decoder.n_layers, 'Encoder and decoder must have equal number of layers'

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        '''
        src                     - [src_len, batch_size]
        trg                     - [trg_len, batch_size]
        teacher_forcing_ratio   - 어느 정도의 teacher forcing, 즉 어느 정도의 ground-truth 를 넣어줄 것인가
        '''
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim # target 의 vocabulary 크기

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        input = trg[0, :] # Decoder 의 첫번째 input 은 <sos> token 이다

        for t in range(1, trg_len):

            '''
            Encoder 에서 나온 hidden, cell state와 input token embedding을 넣어준다
            
            이후 trg_len 까지 새로운 hidden, cell state 를 계속 생성해주면서 output 에 추가해준다
            '''
            output, hiden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1) # predicted token 중 가장 큰 것을 반환

            input = trg[t] if teacher_force else top1 # teacher_force 가 아니라면 그 전 예측값을 넣고 아니면 ground-truth

        return outputs

## Training the Seq2Seq Model

구현된 모델을 학습시키는 과정이다.

지금까지 encoder, decoder, seq2seq 모델의 클래스를 정의해주었으니 이를 선언하고 `cuda`에 올린다

In [27]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
enc_emb_dim = 256
dec_emb_dim = 256
hid_dim = 512
n_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5

enc = Encoder(input_dim=input_dim, emb_dim=enc_emb_dim, hid_dim=hid_dim, n_layers=n_layers, dropout=enc_dropout)
dec = Decoder(output_dim=output_dim, emb_dim=dec_emb_dim, hid_dim=hid_dim, n_layers=n_layers, dropout=dec_dropout)

model = Seq2Seq(enc, dec, device).to(device)

모델 선언 후에 weight initialize를 수행해준다.

`apply` 함수를 통해 쉽게 구현할 수 있고, weight은 uniform distribution을 따르도록 한다.

In [28]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7854, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

모델의 파라미터 수를 세어보자

In [29]:
def count_params(model):
    '''
    모델의 파라미터 중 requires_grad = True, 즉 학습이 수행될 파라미터를

    .numel() 함수로, 개수를 모두 세어준다
    '''
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Number of parameters : {count_params(model)}')

Number of parameters : 13898757


Optimizer와 학습에 사용될 loss function 또한 선언해준다. 

이때 `CrossEntropyLoss`를 사용하는데, padding 된 token은 의미가 없기때문에 loss 계산에서 제외해준다

In [30]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token] # .vocab.stoi 는 현재 단어 집합의 단어와 맵핑된 고유한 정수를 출력해준다

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

이제 학습을 위해 training loop 을 설정한다.

각 iteration은 다음과 같은 과정을 반복한다.

1. Batch로 부터 source, target 문장인 $X, Y$ 를 얻는다

2. gradient 를 초기화해준다 (`optim.zero_grad()`)

3. $X, Y$ 를 모델에 넣어서 $\hat{Y}$ 를 얻는다

4. loss function에 넣고, gradient를 계산한다. (`loss.backward()`)

5. Gradient clip을 통해 exploding 을 막아준다. (RNN 계열에서 많이 사용)

6. Back propagation 을 수행하고, loss를 얻는다.

7. 위의 과정을 하나의 batch에 대해 모두 수행하여 학습 수행

In [31]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()

    epoch_loss = 0

    for i, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        # trg = [trg_len, batch_size]
        # output = [trg_len, batch_size, output_dim]

        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim) # [trg_len, batch_size, output_dim] -> [(trg_len - 1) * batch_size, output_dim]
        trg = trg[1:].view(-1) # [trg_len, batch_size] -> [(trg_len - 1) * batch_size]

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

모델을 evaluation 할 때에는 `model.eval()`을 해야한다. 이를 통해 batch normalization, dropout 등을 사용 안할 수 있다.

또한 `with torch.no_grad()` 를 통해 파라미터들의 gradient 를 계산되지 않게 한다. 이를 통해 메모리 낭비, 속도 등을 개선한다.

마지막으로, 해당 구현에서는 `teacher_forcing_ratio`를 적용하지 않아야 한다.

In [32]:
def evaluate(model, iterator, criterion):
    
    model.eval()

    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) # teacher forcing 없애주기

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

아래의 함수는 한 epoch을 수행하는데 걸리는 시간을 측정하는 함수이다

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

각 epoch 마다 validation set을 넣어서 validation loss를 측정할 수 있다.

In [34]:
n_epochs = 10
clip = 1

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, clip)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '1-seq2seq-model.pt')

    print(f'Epoch : {epoch+1:02} | Time : {epoch_mins}m {epoch_secs}s | Train loss : {train_loss:.5f} | Valid loss : {valid_loss:.5f}')

Epoch : 01 | Time : 0m 20s | Train loss : 5.08197 | Valid loss : 4.93657
Epoch : 02 | Time : 0m 20s | Train loss : 4.57124 | Valid loss : 4.86478
Epoch : 03 | Time : 0m 20s | Train loss : 4.27645 | Valid loss : 4.83725
Epoch : 04 | Time : 0m 20s | Train loss : 4.08189 | Valid loss : 4.70380
Epoch : 05 | Time : 0m 20s | Train loss : 3.93759 | Valid loss : 4.52027
Epoch : 06 | Time : 0m 20s | Train loss : 3.76931 | Valid loss : 4.46131
Epoch : 07 | Time : 0m 20s | Train loss : 3.61954 | Valid loss : 4.24916
Epoch : 08 | Time : 0m 20s | Train loss : 3.44632 | Valid loss : 4.16468
Epoch : 09 | Time : 0m 20s | Train loss : 3.27154 | Valid loss : 4.00326
Epoch : 10 | Time : 0m 20s | Train loss : 3.12329 | Valid loss : 3.91730


학습이 끝난 후에 저장된 모델을 불러와서 test 를 진행할 수 있다

In [35]:
model.load_state_dict(torch.load('1-seq2seq-model.pt'))
test_loss = evaluate(model, test_iterator, criterion)

print(f'Test loss : {test_loss:.5f}')

Test loss : 3.92761
